In [1]:
from pathlib import Path
import json

import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 

# from mat73 import loadmat as loadmat_mat73
# from mat4py import loadmat as loadmat_mat4py

import h5py
from scipy.io import loadmat as loadmat_scipy

# Session analysis
Let's start with a session to look at the files

## Files and their description. A first take

In [2]:
session_path = Path("/home/heberto/buzaki/e13_16f1_210302/")
session_files_path_list = list(session_path.iterdir())
session_files_path_list

[PosixPath('/home/heberto/buzaki/e13_16f1_210302/e13_16f1_210302.SleepState.states.mat'),
 PosixPath('/home/heberto/buzaki/e13_16f1_210302/e13_16f1_210302.ripples.events.mat'),
 PosixPath('/home/heberto/buzaki/e13_16f1_210302/e13_16f1_210302.cell_metrics.cellinfo.mat'),
 PosixPath('/home/heberto/buzaki/e13_16f1_210302/chanMap.mat'),
 PosixPath('/home/heberto/buzaki/e13_16f1_210302/e13_16f1_210302.spikes.cellinfo.mat'),
 PosixPath('/home/heberto/buzaki/e13_16f1_210302/e13_16f1_210302.Behavior.mat'),
 PosixPath('/home/heberto/buzaki/e13_16f1_210302/e13_16f1_210302.mono_res.cellinfo.mat'),
 PosixPath('/home/heberto/buzaki/e13_16f1_210302/e13_16f1_210302.session.mat')]

Now as Cody mention, we have seen some experimental data from this lab already and we are familiar with the names.

In a [previous conversion](https://github.com/catalystneuro/buzsaki-lab-to-nwb/blob/master/buzsaki_lab_to_nwb/yuta_visual_cortex/files_documentation.ipynb) I found out that the matlab files contain the following information:



* `SleepState.states` : This can be considered processed data involving up-down intervals. This can be include as process data.
* `chanMap` : This seems to be concerned with information of the channels in the electrode. For example we find both the x and y coordinates of each of the channels. The structure of the files here is (1, n_channels) where n_channels is 64 for this setup.
* `session` : Contains behavioral info and general information related to the session such as the experimenter, the species, the strain and timestamps for the creation of the session.

As you see, we have the following descriptions missing:
* `.ripples.events.mat`: 
* `Behavior` : 

Which is something that we would do below.

#### Spike sorting
The files related to spike sorting were the following in a previous conversion.

For the previous conversion those were the files related to **cell explorer format / interface**:
* `metric_cell_info`
* `mono_res_cellinfo`
* `spikes.cell_info`

They don't seem equivalent to the ones here. We need to confirm that they are equivalent to the following files in this conversion:
* `cell_metrics.cellinfo.mat`
* `cell_metrics.cellinfo`
* `spikes.cellinfo`

This is done below.


## Exploring some files



### `SleepState.states`

Let's now see now which matlab file opener works best

In [3]:
file_path = session_files_path_list[0]
file_path = file_path.parent / "e13_16f1_210302.SleepState.states.mat"
# Open file_path with loadmat_scipy from scipy
mat_file = loadmat_scipy(file_path, simplify_cells=True)
# Iterate over the keys and print the type of the values
for key in mat_file.keys():
    print(key, type(mat_file[key]))

__header__ <class 'bytes'>
__version__ <class 'str'>
__globals__ <class 'list'>
SleepState <class 'dict'>


This is a recursive structure. Let's print the keys, types and shapes (if numpy array) for exploration

In [4]:

def build_keys_and_types(dictionary):
    output_dict = {}
    for key, value in dictionary.items():
        if isinstance(value, dict):
            output_dict[key] = build_keys_and_types(value)
        elif isinstance(value, np.ndarray):
            if value.size > 10:
                output_dict[key] = {
                    'type': str(type(value)),
                    'shape': str(value.shape)
                }
            else:
                # Print small arrays
                output_dict[key] = {
                    'type': str(type(value)),
                    'value': str(value)
                }
        elif isinstance(value, list):
            if len(value) > 10:
                output_dict[key] = {
                    'type': str(type(value)),
                    'length': len(value)
                }
            else:
                # Print small lists
                output_dict[key] = {
                    'type': str(type(value)),
                    'value': str(value)
                }
        else:
            output_dict[key] = {
                "type": str(type(value)),
                "value": str(value),
            }
    return output_dict

# Define your sleep_state_dict here

result = build_keys_and_types(mat_file)
json_output = json.dumps(result, indent=2)

# Dump to a file in the same folder
json_directory = Path.cwd() / "_json_files"
json_directory.mkdir(exist_ok=True)
with open(json_directory / 'sleep_state_dict.json', 'w') as f:
    f.write(json_output)
    

If you have a matlab licence, you can also just explore the file there. There are two things to look for, large arrays that might
correspond to behavorial data and metadata from the experiment.

Because I know the data from this lab I will be looking for the REM state.



In [5]:
sleep_state_dict = mat_file["SleepState"]
json_dict = json.dumps(build_keys_and_types(sleep_state_dict), indent=4)

In [6]:
sleep_state_dict["ints"]


{'WAKEstate': array([[    1,  1806],
        [ 2947,  3015],
        [ 3314,  3345],
        [ 3995,  4025],
        [ 4525,  5819],
        [ 6221,  6272],
        [ 6834,  6876],
        [ 7481,  7508],
        [ 7614,  7652],
        [ 7879,  7897],
        [ 8111, 14739],
        [15887, 21006]], dtype=uint16),
 'NREMstate': array([[ 1807,  2946],
        [ 3016,  3241],
        [ 3346,  3854],
        [ 4026,  4524],
        [ 5820,  6220],
        [ 6273,  6747],
        [ 6877,  7439],
        [ 7509,  7593],
        [ 7653,  7860],
        [ 7898,  8110],
        [14740, 15810]], dtype=uint16),
 'REMstate': array([[ 3242,  3313],
        [ 3855,  3994],
        [ 6748,  6833],
        [ 7440,  7480],
        [ 7594,  7613],
        [ 7861,  7878],
        [15811, 15886]], dtype=uint16)}

In [7]:
wake_state = sleep_state_dict["ints"]["WAKEstate"]
wake_state  # This is the start time and the end time of the wake state. Probably in frames.

array([[    1,  1806],
       [ 2947,  3015],
       [ 3314,  3345],
       [ 3995,  4025],
       [ 4525,  5819],
       [ 6221,  6272],
       [ 6834,  6876],
       [ 7481,  7508],
       [ 7614,  7652],
       [ 7879,  7897],
       [ 8111, 14739],
       [15887, 21006]], dtype=uint16)

So, we need to confirm the units and this can go as a `TimeIntervals` in a processing module

### `Behavior`

In [8]:
file_path = file_path.parent / "e13_16f1_210302.Behavior.mat" 
file_path.is_file()

True

In [9]:
mat_file = loadmat_scipy(file_path, simplify_cells=True)

# Output to an external file
with open(json_directory / 'behavior_dict.json', 'w') as f:
    f.write(json.dumps(build_keys_and_types(mat_file), indent=4))

Importantly this contains a date in `date` and subject information in `animal`.
Surprisingly, this does not seem to contain any large vector as I expected

### `ripples.events.mat`

In [10]:
file_path = session_files_path_list[0]
file_path = file_path.parent / "e13_16f1_210302.ripples.events.mat"
file_path.is_file()

True

In [11]:
mat_file = loadmat_scipy(file_path, simplify_cells=True)

# Output to an external file
with open(json_directory / 'riples.json', 'w') as f:
    f.write(json.dumps(build_keys_and_types(mat_file), indent=4))

This is a large file. We need to look into the paper to see what should be stored from here

### `chanMap.mat`

In [12]:
file_path = session_files_path_list[0]
file_path = file_path.parent / "chanMap.mat'"
file_path.is_file()


False

In [13]:
# Write this to a file for visualization
with open(json_directory / 'chanMap.json', 'w') as f:
    f.write(json.dumps(build_keys_and_types(mat_file), indent=4))

There are some time series here but no information about the channels as I was expeting.

### `Session`

In [14]:
file_path = session_files_path_list[0]
file_path = file_path.parent / "e13_16f1_210302.session.mat"
file_path.is_file()


True

In [15]:
mat_file = loadmat_scipy(file_path, simplify_cells=True)

# Output to an external file
with open(json_directory / 'session.json', 'w') as f:
    f.write(json.dumps(build_keys_and_types(mat_file), indent=4))

This contains useful information like subject, animal and an epoch file.
Probably less relevant is information about other sources of data such as spikesorting and some of the analogous channels.

# Spikesorting 

## Testing CellExplorer

In [16]:
from spikeinterface.extractors import CellExplorerSortingExtractor

file_path = session_files_path_list[0].parent / "e13_16f1_210302.spikes.cellinfo.mat"
extractor = CellExplorerSortingExtractor(file_path)

/home/heberto/miniconda3/envs/neuroconv_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AssertionError: No e13_16f1_210302.sessionInfo.mat file found in the folder!

No `sessionInfo` for this file so the above throws an assertion. Let's see what each of the files.

The `CellExplorerSortingExtractor` uses the `sessionInfo` to extract the sampling frequency. But that might be somehwere else. 

Importantly, the files should contain the fields `UID` and `times` in a field called `spikes`. Let's see if any of the files contain this information and if it is consistent across them

Looking at the files below, they do seem to agree with the basic information. It should be straighforward to role a new sorting extractor for this dataset. Or use NumpySortingExtractor and then add the `sessionInfo` manually. Not sure at this point on what it would be easier. 

## Individual spike files

### `spikes.cellinfo.mat`

In [ ]:
file_path = session_files_path_list[0].parent / "e13_16f1_210302.spikes.cellinfo.mat"

mat_file = loadmat_scipy(file_path, simplify_cells=True)

# Output to an external file
with open(json_directory / 'cellinfo.json', 'w') as f:
    f.write(json.dumps(build_keys_and_types(mat_file), indent=4))

In [ ]:
spikes = mat_file["spikes"]
spikes.keys()

In [ ]:
sampling_rate = spikes["sr"]
cluster_id = spikes["cluID"]
times = spikes["times"]
unit_ids = spikes["UID"]

sampling_rate, cluster_id.shape, times.shape, unit_ids.shape, unit_ids[:3], cluster_id[:3], times[:3][0][:3]


### `cell_metrics.cellinfo.mat`

In [ ]:
file_path = session_files_path_list[0].parent / "e13_16f1_210302.cell_metrics.cellinfo.mat"

# Ouput to an external file
with open(json_directory / 'cell_metrics.json', 'w') as f:
    f.write(json.dumps(build_keys_and_types(mat_file), indent=4))

In [ ]:
spikes = mat_file["spikes"]
spikes.keys()

In [ ]:
sampling_rate = spikes["sr"]
cluster_id = spikes["cluID"]
times = spikes["times"]
unit_ids = spikes["UID"]

sampling_rate, cluster_id.shape, times.shape, unit_ids.shape, unit_ids[:3], cluster_id[:3], times[:3][0][:3]


### `mono_ress.cellinfo.mat`

In [ ]:
file_path = session_files_path_list[0].parent / "e13_16f1_210302.mono_res.cellinfo.mat"

# Output to an external file
with open(json_directory / 'mono_res.json', 'w') as f:
    f.write(json.dumps(build_keys_and_types(mat_file), indent=4))
    


In [ ]:
spikes = mat_file["spikes"]
spikes.keys()

In [ ]:
sampling_rate = spikes["sr"]
cluster_id = spikes["cluID"]
times = spikes["times"]
unit_ids = spikes["UID"]

sampling_rate, cluster_id.shape, times.shape, unit_ids.shape, unit_ids[:3], cluster_id[:3], times[:3][0][:3]
